In [ ]:
from init_notebook import *

In [ ]:
from src.clipig.transformations.value_trans import Denoising
print(next(filter(lambda p: p["name"] == "model", Denoising.PARAMS))["choices"])

In [ ]:
denoiser = Denoising(
    #model="degradient-mid-64x64-150k",
    #model="degradient-strong-64x64-230k",
    #model="denoise-mid-64x64-150k",
    #model="declip-1",
    model="denoise-heavy-2",
    mix=1.,
)
denoiser

In [ ]:
@torch.no_grad()
def denoise_loop(
    image: torch.Tensor,
):
    images = []
    for i in tqdm(range(8*8)):
        image = image - image.min()
        image = image / image.max()
        #print(image.min(), image.max())

        image = denoiser.model(image.unsqueeze(0))[0]
        images.append(image)
        image = (image * 1.01 - 0.01)
        #image = image + .8 * torch.randn_like(image)
        image = image.clamp(0, 1)

    display(VF.to_pil_image(make_grid(images)))

denoise_loop(
    #torch.rand(3, 128, 128)
    torch.randn(3, 128, 128).abs()
)


## conditional

In [ ]:
from experiments.denoise.resconv_cond import ConditionalResConv

kernel_size = []
padding = []
for i in range(9):
    t = i / (9 - 1)
    ks = 3 * (1. - t) + t * 9
    ks = int(ks / 2) * 2 + 1
    kernel_size.append(ks)
    padding.append(int(math.floor(ks / 2)))

class Module(nn.Module):
    def __init__(self):
        super().__init__()
        self.module = ConditionalResConv(
            in_channels=3,
            condition_size=30,
            num_layers=9,
            channels=32,
            stride=1,
            kernel_size=kernel_size,
            padding=padding,
            activation="gelu",
            activation_last_layer=None,
        )

    def forward(self, x, condition):
        return (x - self.module(x, condition)).clamp(0, 1) 

model = Module().eval()
data = torch.load(
    #"../checkpoints/denoise/conresconv-denoise_ds-mnist_l-9_ks1-3_ks2-9_ch-32_stride-1_act-gelu/snapshot.pt"
    #"../checkpoints/denoise/conresconv-denoise_ds-mnist_noise-0,1.1_l-9_ks1-3_ks2-9_ch-32_stride-1_act-gelu/snapshot.pt",
    
    #"../checkpoints/denoise/conresconv-denoise_ds-pix_noise-0.5,1.0_l-9_ks1-3_ks2-9_ch-32_stride-1_act-gelu/snapshot.pt"
    #"../checkpoints/denoise/conresconv-difnoise_ds-pix_noise-0.1,1.0_l-9_ks1-3_ks2-9_ch-32_stride-1_act-gelu/snapshot.pt"
    "../checkpoints/denoise/conresconv-difnoise_ds-pix_noise-0.1,1.3_l-9_ks1-3_ks2-9_ch-32_stride-1_act-gelu/snapshot.pt"
)
print(data["num_input_steps"], "steps")
model.load_state_dict(data["state_dict"])

In [ ]:
@torch.no_grad()
def denoise_conditional_loop(
    image_batch: torch.Tensor,
    conditionals: torch.Tensor,
    frames: int = 20,
    frame_stride: int = 1,
):
    images = list(image_batch)
    try:
        for i in tqdm(range(frames * frame_stride)):
            #image_batch = image_batch - image_batch.min()
            #image_batch = image_batch / image_batch.max()
            #print(image_batch.min(), image_batch.max())
            
            image_batch = model(image_batch, conditionals)
            #image_batch += .5 * (image_batch_d - image_batch)
            if i % frame_stride == 0:
                for image in image_batch:
                    images.append(image)

            image_batch = .5 + (image_batch - image_batch.mean()) * 1.1
            #image_batch = (image_batch * 1.02 - 0.01)
            image_batch = image_batch + .1 * torch.randn_like(image_batch)
            
            image_batch = image_batch.clamp(0, 1)
    except KeyboardInterrupt:
        pass
        
    display(VF.to_pil_image(make_grid(images, nrow=image_batch.shape[0])))

N=30
denoise_conditional_loop(
    #torch.rand(3, 128, 128)
    (.5 + 2*torch.randn(N, 3, 32, 32)).clamp(0, 1),
    
    torch.diag(torch.Tensor([1] * N))[:30],
    #torch.Tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
    frame_stride=1, frames=20,
)
